In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#need to update train_#_content.csv

# Load processed data

In [ ]:
content_df=pd.read_csv('/kaggle/input/riiid-contents/content_new.csv')
content_df=content_df[['content_id','part','type_of_encoding','tag_ids','content_accuracy']]
#use for embedding layers
content_ids = content_df.content_id.max() + 2
part_ids = content_df.part.max() +1
types = content_df.type_of_encoding.max() + 1
tag_ids = content_df.tag_ids.max() + 1
task_container_ids = 10001

In [ ]:
# pd.read_csv('/kaggle/input/riiid-content`s/train_2_new(jjin).csv', nrows=10**6)

In [ ]:
dtype = {
    'answered_correctly': 'int8',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
#     'task_container_id': 'int16',
    'prior_question_elapsed_time': 'float32',
#     'prior_question_had_explanation': 'int8',
    'part': 'int8',
    'tag_ids': 'int16',
#     'type_of_encoding': 'int8',
    'content_accuracy': 'float16',
}

#Load Train data
df = pd.read_csv(
    '/kaggle/input/riiid-contents/train_2_new(jjin).csv',
    usecols=dtype.keys(),
    dtype=dtype,
    nrows=2*10**6
)

In [ ]:
df = {uid: u.drop(columns='user_id') for uid, u in df.groupby('user_id')}

In [ ]:
#Load Train data
df2 = pd.read_csv(
    '/kaggle/input/riiid-contents/train_1_new(jjin).csv',
    usecols=dtype.keys(),
    dtype=dtype,
    nrows=2*10**5
)
df2 = {uid: u.drop(columns='user_id') for uid, u in df2.groupby('user_id')}
# df.update(df2)

In [ ]:
len(df)

In [ ]:
# #Load Train data
# df2 = pd.read_csv(
#     '/kaggle/input/riiid-contents/train_3_new(jjin).csv',
#     usecols=dtype.keys(),
#     dtype=dtype,
# #     nrows=3*10**6
# )
# df2 = {uid: u.drop(columns='user_id') for uid, u in df2.groupby('user_id')}
# print(len(df2))
# df.update(df2)

In [ ]:
len(df)

In [ ]:
for k in df.keys():
    v = df[k]
    if len(v)>2000:
        df[k]=df[k][:2000]

In [ ]:
for k in df2.keys():
    v = df2[k]
    if len(v)>2000:
        df2[k]=df2[k][:2000]

In [ ]:
import tensorflow as tf

In [ ]:
# just some stuff I ctrl C ctrl V from StackOverflow (with little changes)
# [1,2,3,4] --- w = 2 --[[1,2], [2,3], [3,4]] but 2D to 3D
def rolling_window(a, w):
    s0, s1 = a.strides
    m, n = a.shape
    return np.lib.stride_tricks.as_strided(
        a, 
        shape=(m-w+1, w, n), 
        strides=(s0, s0, s1)
    )


def make_time_series(x, windows_size):
  x = np.pad(x, [[ windows_size-1, 0], [0, 0]], constant_values=0)
  x = rolling_window(x, windows_size)
  return x


def add_features_to_user(user):
    # We add one to the column in order to have zeros as padding values
    # Start Of Sentence (SOS) token will be 3. 
    user['answered_correctly'] = user['answered_correctly'].shift(fill_value=2)+1
    return user

In [ ]:
class RiidSequence(tf.keras.utils.Sequence):

  def __init__(self, 
               users, 
               windows_size,
               batch_size=64,
               start=0,
               end=None):
    self.users = users # {'user_id': user_df, ...} Dictionary
    self.windows_size = windows_size
    # to convert indices to our keys
    self.mapper = dict(zip(range(len(users)), users.keys()))
    # start and end to easy generate training and validation
    self.start = start
    self.end = end if end else len(users)
    # To know where the answered_correctly_column is
    self.answered_correctly_index = list(self.user_example().columns).index('answered_correctly')
    
    ## user dict for feature 
#     self.user_history={}
        
  def __len__(self):
    return self.end-self.start

  # get df from dict and generate numpy
  def __getitem__(self, idx):
    uid = self.mapper[idx+self.start]
    user = self.users[uid].copy()
    y = user['answered_correctly'].to_numpy().copy()
    x = add_features_to_user(user)
    return make_time_series(x, self.windows_size), y

  
  def user_example(self):
    """Just to check what we have till' now."""
    uid = self.mapper[self.start]
    return add_features_to_user(self.users[uid].copy())



  # INFERENCE PART    
  def get_user_for_inference(self, user_row):
    """Picks a new user row and concats it to previous interactions 
    if it was already stored.
    
    Maybe the biggest trick in the notebook is here. We reuse the user_id column to 
    insert the answered_correctly SOS token because we previously placed the column 
    there on purpose.
    
    After it, we roll that column and then crop it if it was bigger than the window
    size, making the SOS token disapear if out of the sequence.
    
    If the sequence if shorter than the window size, then we pad it.
    """
    
    uid = user_row[self.answered_correctly_index]
    
    user_row[self.answered_correctly_index] = 2 # SOS token
    user_row = user_row[np.newaxis, ...]
    
    if uid in self.users:
      x = np.concatenate([self.users[uid], user_row])
      # same as in training, we need to add one!!!
      x[:, self.answered_correctly_index] = np.roll(x[:, self.answered_correctly_index], 1) + 1
    else:
      x = user_row
     
    if x.shape[0] < self.windows_size:
      return np.pad(x, [[self.windows_size-x.shape[0], 0], [0, 0]])
    elif x.shape[0] > self.windows_size:
      return x[-self.windows_size:]
    else:
      return x

  def update_user(self, uid, user):
    """Concat the new user's interactions to the old ones if already stored."""
    if uid in self.users:
      self.users[uid] = \
        np.concatenate([self.users[uid], user])[-self.windows_size:]
    else:
      self.users[uid] = user

In [ ]:
RiidSequence(df, 64).user_example().head()

In [ ]:
# x, y = RiidSequence(df, 4)[2]
# x.shape, y.shape

In [ ]:
# POSITION ENCODING

def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

# NN THINGS
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpose_b=True)
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
  output = tf.matmul(attention_weights, v)
  return output, attention_weights

    
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    q = self.wq(q)
    k = self.wk(k)
    v = self.wv(v)

    q = self.split_heads(q, batch_size)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))

    output = self.dense(concat_attention)

    return output, attention_weights


def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])


class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask) 
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output) 

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output) 

    return out2

In [ ]:
def create_padding_mask(seqs):
  # We mask only those vectors of the sequence in which we have all zeroes 
  # (this is more scalable for some situations).
  mask = tf.cast(tf.reduce_all(tf.math.equal(seqs, 0), axis=-1), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
columns = list(RiidSequence(df, 64).user_example().columns)
columns

In [ ]:
df[list(df.keys())[0]]

In [ ]:
def get_series_model(
        n_features,
        content_ids,
        task_container_ids,
        part_ids,
        windows_size=64,
        d_model=24,
        num_heads=4,
        n_encoder_layers = 2
    ):
    # Input
    inputs = tf.keras.Input(shape=(windows_size, n_features), name='inputs')
    mask = create_padding_mask(inputs)
    pos_enc = positional_encoding(windows_size, d_model)    
    
    # Divide branches
    content_id = inputs[..., 0]
#     content_type_id = inputs[..., 1] ## ADD
#     task_container_id = inputs[..., 2]
    answered_correctly = inputs[..., 1]
    elapsed_time = inputs[..., 2]
#     had_explanation = inputs[..., 5] ## ADD
    part = inputs[..., 3]
    tag_id = inputs[..., 4]
#     type_of = inputs[..., 8]
    acc = inputs[..., 5]
    
    acc = tf.expand_dims(acc, axis=-1, name=None)
    elapsed_time = tf.expand_dims(elapsed_time, axis=-1, name=None)
    
     # Create embeddings
    content_embeddings = tf.keras.layers.Embedding(content_ids, d_model)(content_id)
#     content_type_embeddings = tf.keras.layers.Embedding(3, d_model)(content_type_id) ## ADD
#     task_embeddings = tf.keras.layers.Embedding(task_container_ids, d_model)(task_container_id)
    answered_correctly_embeddings = tf.keras.layers.Embedding(4, d_model)(answered_correctly)
#     had_explanation_embeddings = tf.keras.layers.Embedding(3, d_model)(had_explanation) ## ADD
    
    tag_id_embeddings = tf.keras.layers.Embedding(tag_ids, d_model)(tag_id)
#     type_of_embeddings = tf.keras.layers.Embedding(types, d_model)(type_of)
    
    # Continuous! Only a learnable layer for it.
#     elapsed_time_embeddings = tf.keras.layers.Dense(d_model, use_bias=False)(elapsed_time)
    part_embeddings = tf.keras.layers.Embedding(part_ids, d_model)(part)
#     acc_embeddings = tf.keras.layers.Dense(d_model, use_bias=False)(acc)
    
    
    # Add embeddings
    x = tf.keras.layers.Add()([
        pos_enc,
        content_embeddings,
#         content_type_embeddings, ## ADD
#         task_embeddings,
        answered_correctly_embeddings,
#         had_explanation_embeddings, ## ADD
#         elapsed_time_embeddings,
        part_embeddings,
        tag_id_embeddings,## ADD
#         type_of_embeddings,## ADD
#         acc_embeddings, ## ADD
        
    ])

    for _ in range(n_encoder_layers):
        x = EncoderLayer(d_model=d_model, num_heads=num_heads, dff=d_model*4, rate=0.1)(x, mask=mask)
    
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x)
    acc1 = tf.keras.layers.GlobalAveragePooling1D()(acc)
    et1 = tf.keras.layers.GlobalAveragePooling1D()(elapsed_time)
    
    x2 = tf.keras.layers.GlobalMaxPooling1D()(x)
    acc2 = tf.keras.layers.GlobalMaxPool1D()(acc)
    et1 = tf.keras.layers.GlobalMaxPool1D()(elapsed_time)
    
    
    x1 = tf.concat([x1,acc1], axis=-1, name='concat')
    x2 = tf.concat([x2,acc2], axis=-1, name='concat2')
#     print(x.shape)
    x1 = tf.keras.layers.Dropout(0.4)(x1)
    x1 = tf.keras.layers.Dense(4, activation='relu')(x1)
    
    x2 = tf.keras.layers.Dropout(0.4)(x2)
    x2 = tf.keras.layers.Dense(4, activation='relu')(x2)
    
    x = tf.concat([x1,x2], axis=-1)
#     print(x.shape)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
    return tf.keras.Model(inputs, output, name='model')

In [ ]:
train_idx = int(len(df)*0.99)
windows_size = 64
epochs = 5
patience = 2
d_model = 16
num_heads = 4
n_encoder_layers = 2

In [ ]:
s_train = RiidSequence(df, windows_size, start=0, end=train_idx)
# s_val = RiidSequence(df, windows_size, start=train_idx)
s_val = RiidSequence(df2, windows_size)

In [ ]:
s_train[0][0][0].shape

In [ ]:
n_features = s_train[0][0].shape[-1]

tf.keras.backend.clear_session()
model = get_series_model(
        n_features,
        content_ids,
        task_container_ids,
        part_ids,
        windows_size=windows_size,
        d_model=d_model,
        num_heads=num_heads,
        n_encoder_layers=n_encoder_layers
    )

optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
)

model.compile(
    optimizer= optimizer, 
    loss='binary_crossentropy', 
    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy(name='acc')]
)

In [ ]:
model.summary()

In [ ]:
# tf.keras.utils.plot_model(model)
import gc
gc.collect()

In [ ]:
model.fit(
    s_train,
    validation_data=s_val,
    epochs=epochs,
#     workers=32,
    shuffle=True,
#     use_multiprocessing=True,
    callbacks=tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_AUC', mode='max', restore_best_weights=True),
    verbose=1
)
model.save_weights('model_2.h5')

In [ ]:
# model.load_weights('/kaggle/input/weight/model_2.h5')

# Prediction

In [ ]:
import gc
# del s_val
# del s_train
gc.collect()

In [ ]:
content_df.columns

In [ ]:
columns

In [ ]:
# answered_correctly  --> user_id
columns[columns.index('answered_correctly')] = 'user_id'
columns = ['content_id'] + [c for c in columns if c not in content_df.columns] + ['row_id'] 
columns

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
#prepare for processing test data
with open('/kaggle/input/riiid-contents/lecture_id.pickle', 'rb') as handle:
    lecture_id = pickle.load(handle)


# max_elaped = max(df['prior_question_elapsed_time'])
# max_elaped = 30000    
    
# Function for testset preprocess
def preprocess_dataset(df, content_df, lecture_id):
    
    # dataset content_id switch
    def change_lecture_id(x):
        content_id, lecture = x[0], x[1]
        if content_id in lecture_id and lecture==1:
            return lecture_id[content_id]
        else:
            return content_id
        
    df = df.fillna(0)    
    df.prior_question_had_explanation = df.prior_question_had_explanation.apply(int) 
    # lecture_id switch axis=1
    df[['content_id']] = df[['content_id','content_type_id']].apply(change_lecture_id,axis=1)
#     df.answered_correctly.replace({-1: 1}, inplace=True)
    
    #fill nan & normalize
    df['prior_question_elapsed_time'] = df['prior_question_elapsed_time'].astype(np.float32)/max(df['prior_question_elapsed_time'])
    
    # dataset-content_id merge
    df = pd.merge(df,content_df[['content_id', 'part', 'tag_ids', 'type_of_encoding']], on='content_id')
    
    return df

In [ ]:
for test_, sample_prediction in iter_test:
    
    try:
        prior_correct = eval(test['prior_group_answers_correct'].iloc[0]) #이전 그룹의 정답여부 
        prior_correct = [a for a in prior_correct if a != -1] # -1 이 아닌경우만
    except:
        prior_correct = []
    
    # Add prior correct to test and update stored users
    if prior_correct:
        prior_test.insert(s_train.answered_correctly_index, 'answered_correctly', prior_correct)
        for uid, user in prior_test.groupby('user_id'):
            s_train.update_user(uid, user.drop(columns='user_id').to_numpy())    

    test = test_.loc[
        test_['content_type_id'] == 0,
        columns
    ]
    
    # Add global features
    test = preprocess_dataset(test, content_df, lecture_id)

    # Save test for later
    prior_test = test.drop(columns='row_id').copy()

    # Make x
    x = np.apply_along_axis(
        s_train.get_user_for_inference,
        1,
        test.drop(columns='row_id').to_numpy()
    )
    
    # Predict
    test['answered_correctly'] = model.predict(x, batch_size=x.shape[0])
    env.predict(test.loc[test['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
test.loc[test['content_type_id'] == 0, ['row_id', 'answered_correctly']]

In [ ]:
test_.columns

In [ ]:
columns

In [ ]:
test = test_.loc[
    test_['content_type_id'] == 0,
    columns
]
test

In [ ]:
preprocess_dataset(test, content_df, lecture_id)

In [ ]:
test.drop(columns='row_id')

In [ ]:
    content_id = inputs[..., 0]
    content_type_id = inputs[..., 1] ## ADD
    task_container_id = inputs[..., 2]
    answered_correctly = inputs[..., 3]
    elapsed_time = inputs[..., 4]
    had_explanation = inputs[..., 5] ## ADD
    part = inputs[..., 6]
    
    tag_id = inputs[..., 7]
    type_of = inputs[..., 8]

In [ ]:
test = test_[columns]

# Add global features
test = preprocess_dataset(test, content_df, lecture_id)

# Save test for later
prior_test = test.drop(columns='row_id').copy()

# Make x
x = np.apply_along_axis(
    s_train.get_user_for_inference,
    1,
    test.drop(columns=['row_id']).to_numpy()
)

# Predict
print(x.shape)
test['answered_correctly'] = model.predict(x, batch_size=x.shape[0])
env.predict(test.loc[test['content_type_id'] == 0, ['row_id', 'answered_correctly']])

In [ ]:
result = test.loc[test['content_type_id'] == 0, ['row_id', 'answered_correctly']]

In [ ]:
result

In [ ]:
result['answered_correctly'] = (result['answered_correctly'] + result['answered_correctly']*1.4)/2
result